In [1]:
import numpy as np
import pandas as pd
import math
import scipy
import matplotlib.pyplot as plt
from scipy.stats import norm,binom
from scipy.io import loadmat
from scipy.special import digamma, gammaln, gamma
from numpy.linalg import inv
#import cv2
from scipy import ndimage
import tensorflow as tf
import os, sys

C:\Users\keert\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
kernel = loadmat('kernel1.mat')

In [4]:
kernel['A'].shape


(48, 48)

In [1]:
from utils.data_load import Image_data

In [2]:
image_data = Image_data(npyfilename = ['gray_scale.npy','PASCAL_data.npy'],
                        kernel_file_name = 'kernel1.mat')

loaded VOC data inVOCdevkit/VOC2012/JPEGImages/
loaded VOC data inVOCdevkit/VOC2007/JPEGImages/
original images loaded


In [5]:
PASCAL_data = image_data.original_images

In [7]:
import numpy as np
np.save('PASCAL_data.npy',PASCAL_data)

In [ ]:
image_data.compute_image_conv()

In [ ]:
plt.imshow(orig_images[27000],cmap = 'gray')
plt.show()
